In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math
import geemap

from IPython.display import Image

import folium
from folium import plugins
import importlib

import geopandas as gpd
from PIL import Image

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWhjYtKG2pNKmlmEluFsUpZZzJjbiTDuCSsYLBcgyIB3SIf0IYIHuGo

Successfully saved authorization token.


In [3]:
ee.Initialize()

*** Earth Engine *** Please upgrade your Earth Engine Python client: authentication is changing! https://developers.google.com/earth-engine/guides/python_install


In [29]:
col_sent = ee.ImageCollection("COPERNICUS/S2_SR")
counties = ee.FeatureCollection('TIGER/2018/Counties')

In [30]:
#print(counties.propertyNames())
#print(counties.get('NAME'))
#print(counties.first())

#col_sent

In [31]:
# Apply filter where country name equals Uganda.

cookCounty = counties.filter(ee.Filter.eq("NAME", 'Cook'))
print(cookCounty)
#tn = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq("NAME", 'Tennessee'))

#Map.centerObject(tn, 6)
#Map.addLayer(tn, {}, 'Tennessee')

#Map.addLayerControl()
#Map

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "TIGER/2018/Counties"
            }
          }
        }
      },
      "filter": {
        "functionInvocationValue": {
          "functionName": "Filter.equals",
          "arguments": {
            "leftField": {
              "constantValue": "NAME"
            },
            "rightValue": {
              "constantValue": "Cook"
            }
          }
        }
      }
    }
  }
})


In [32]:
# Print new "ugandaBorder" object and explorer features and properties.
# There should only be one feature representing Uganda.
#print(ChicagoBorder)

# Add Uganda outline to the Map as a layer.
Map.centerObject(ChicagoBorder, 6)
Map.addLayer(ChicagoBorder)

NameError: name 'Map' is not defined

In [ ]:
# Subset images with less than 5% cloud cover, then subset those which intersect the 
# lower left corner of the rectangle 'poly' we've just defined above.
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 
                                      5)).filterBounds(poly)

# Adjust chosen bands to anything you want, and the featurization will flow from it
chosen_bands = ['B2', 'B3', 'B4', 'B8']

# choose only desired bands from the image 
bands = clouds.select(chosen_bands)
s = bands.sort('CLOUD_COVER')
# s contains all sentinel images which satisfied the above two filters 

# Here you  choose whether you want to have:
#     (1) the least cloudy image s.first()
#     (2) the median of all images in s 


# Option 1:
#scene = s.first()

# Option 2:
scene = s.reduce(ee.Reducer.median())

In [ ]:
# Applying the same filters as above, but only to the Near Infared Band (NIR):
nir = clouds.select(['B8']).reduce(ee.Reducer.median())

# Do the same for the red band
red = clouds.select(['B4']).reduce(ee.Reducer.median())

In [ ]:
# Calculate NDVI (Normalized Difference Vegetation Index):
numer = nir.add(red)
denom = nir.subtract(red)

ndvi = denom.divide(numer)

In [ ]:
// Define a point of interest. Use the UI Drawing Tools to import a point
// geometry and name it "point" or set the point coordinates with the
// ee.Geometry.Point() function as demonstrated here.
var point = ee.Geometry.Point([-122.292, 37.9018]);

// Import the Landsat 8 TOA image collection.
var l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA');

// Get the least cloudy image in 2015.
var image = ee.Image(
  l8.filterBounds(point)
    .filterDate('2015-01-01', '2015-12-31')
    .sort('CLOUD_COVER')
    .first()
);

In [ ]:
// Compute the Normalized Difference Vegetation Index (NDVI).
var nir = image.select('B5');
var red = image.select('B4');
var ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');

// Display the result.
Map.centerObject(image, 9);
var ndviParams = {min: -1, max: 1, palette: ['blue', 'white', 'green']};
Map.addLayer(ndvi, ndviParams, 'NDVI image');

In [ ]:
var ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');

In [ ]:
var addNDVI = function(image) {
  var ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvi);
};

// Test the addNDVI function on a single image.
var ndvi = addNDVI(image).select('NDVI');